## Example of training/testing using deterministic models with pytorch-lightning

In [1]:
%load_ext lab_black

In [2]:
import numpy as np
import pandas as pd
from pathlib import Path
from types import SimpleNamespace
import pytorch_lightning as pl
from pytorch_lightning.callbacks import EarlyStopping, ModelCheckpoint
from bnnrul.cmapss.dataset import CMAPSSDataModule
from bnnrul.cmapss.models import CMAPSSModel, get_checkpoint

In [3]:
args = SimpleNamespace(
    data_path="../data/cmapss/",
    out_path="../results/cmapss/",
    scn="deterministic",
    arch="linear",
)

In [4]:
data = CMAPSSDataModule(args.data_path, batch_size=1000)
model = CMAPSSModel(data.win_length, data.n_features, args.arch)

checkpoint_dir = Path(f"{args.out_path}/{args.scn}/checkpoints/{args.arch}")
checkpoint_file = get_checkpoint(checkpoint_dir)
monitor = "loss/val"
checkpoint_callback = ModelCheckpoint(dirpath=checkpoint_dir, monitor=monitor)
earlystopping_callback = EarlyStopping(monitor=monitor, patience=20)

trainer = pl.Trainer(
    gpus=[0],
    max_epochs=2,
    log_every_n_steps=10,
    logger=pl.loggers.TensorBoardLogger(
        f"{args.out_path}/{args.scn}/lightning_logs/{args.arch}"
    ),
    callbacks=[
        checkpoint_callback,
        earlystopping_callback,
    ],
)
trainer.fit(model, data)
print("Best model: ", checkpoint_callback.best_model_path)
_ = Path(
    f"{args.out_path}/{args.scn}/checkpoints/{args.arch}/best_model_path.txt"
).write_text(f"{checkpoint_callback.best_model_path}")

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
Missing logger folder: ../results/cmapss//deterministic/lightning_logs/linear/lightning_logs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2]

  | Name   | Type   | Params
----------------------------------
0 | layers | Linear | 45.6 K
----------------------------------
45.6 K    Trainable params
0         Non-trainable params
45.6 K    Total params
0.182     Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Best model:  /home/luis/repos/bnnrul/results/cmapss/deterministic/checkpoints/linear/epoch=1-step=32.ckpt


In [5]:
data = CMAPSSDataModule(args.data_path, batch_size=10000)
model = CMAPSSModel.load_from_checkpoint(
    Path(
        f"{args.out_path}/{args.scn}/checkpoints/{args.arch}/best_model_path.txt"
    ).read_text(),
)
trainer = pl.Trainer(
    gpus=[0],
    log_every_n_steps=10,
    logger=pl.loggers.TensorBoardLogger(
        f"{args.out_path}/{args.scn}/lightning_logs/{args.arch}"
    ),
)
trainer.test(model, data, verbose=False)

/home/luis/miniconda3/envs/bnnrul/lib/python3.9/site-packages/pytorch_lightning-1.6.0rc1-py3.9.egg/pytorch_lightning/loops/utilities.py:91: PossibleUserWarning: `max_epochs` was not set. Setting it to 1000 epochs. To train without an epoch limit, set `max_epochs=-1`.
  rank_zero_warn(
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2]


Testing: 0it [00:00, ?it/s]

/home/luis/miniconda3/envs/bnnrul/lib/python3.9/site-packages/bnnrul-0.1-py3.9.egg/bnnrul/cmapss/models.py:103: UserWarning: Using a target size (torch.Size([10000])) that is different to the input size (torch.Size([10000, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
/home/luis/miniconda3/envs/bnnrul/lib/python3.9/site-packages/bnnrul-0.1-py3.9.egg/bnnrul/cmapss/models.py:103: UserWarning: Using a target size (torch.Size([1396])) that is different to the input size (torch.Size([1396, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.


[{'y_0': -20.67729949951172,
  'y_hat0': 3.8339343070983887,
  'err_0': 60.00567626953125,
  'y_1': -31.992835998535156,
  'y_hat1': 3.833934783935547,
  'err_1': 66.37532806396484}]